<a href="https://colab.research.google.com/github/joy-narula/chatbot-langchain/blob/main/FineTuning_DPO_gemma.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch

major_version, minor_version = torch.cuda.get_device_capability()
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
if major_version >= 8:
  !pip install --no-deps packaging ninja einops flash-attn xformers trl peft accelerate bitsandbytes
else:
  !pip install --no-deps xformers trl peft accelerate bitsandbytes

  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-install-h4g0444r/unsloth_c0e2dc690a05479ba90c71d616df055e
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-install-h4g0444r/unsloth_c0e2dc690a05479ba90c71d616df055e
  Resolved https://github.com/unslothai/unsloth.git to commit 9dc399a6b6625ee40835c5eab361426d3c5d4abb
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 MB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 22.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 37.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.2/60.2 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 31.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.1/112.1 kB 9.8 MB/s eta 0:00:00
   ━━

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.7/16.7 MB 62.7 MB/s eta 0:00:00


In [2]:
from unsloth import FastLanguageModel
from trl import SFTTrainer
from transformers import AutoTokenizer, TrainingArguments
from datasets import load_dataset

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


In [3]:
max_seq_length = 2048
dtype = None
load_in_4bit = True

In [ ]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/gemma-7b-bnb-4bit",
    max_seq_length=max_seq_length,
    dtype=dtype,
    load_in_4bit=load_in_4bit,
)

tokenizer = AutoTokenizer.from_pretrained("philschmid/gemma-tokenizer-chatml", use_fast=True
                                          )

==((====))==  Unsloth 2024.12.2: Fast Gemma patching. Transformers:4.46.2.
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu121. CUDA: 7.5. CUDA Toolkit: 12.1. Triton: 3.1.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.28.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.57G [00:00<?, ?B/s]

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    lora_alpha = 16,
    lora_dropout = 0,
    bias="none",
    use_gradient_checkpointing=True,
    random_state= 3407
)

In [ ]:
dataset = load_dataset("HuggingFaceH4/deita-10k-v0-sft", split=['train_sft', 'test_sft'])

In [ ]:
training_args = TrainingArguments(
    do_eval = True,
    evaluation_strategy = "steps",
    eval_steps = 100,
    save_strategy = "epoch",
    per_device_train_batch_size = 4,
    per_device_eval_batch_size = 4,
    gradient_accumulation_steps = 4,
    warmup_ratio = 0.1,
    num_train_epochs = 1,
    learning_rate = 2.0e-05,
    fp16 = not torch.cuda.is_bf16_supported(),
    bf16 = torch.cuda.is_bf16_supported(),
    logging_steps = 100,
    optim = "adamw_8bit",
    lr_scheduler_type = "cosine",
    seed = 3507,
    output_dir = "./"
)

In [ ]:
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset[0],
    eval_dataset = dataset[1],
    max_seq_length = max_seq_length,
    dataset_kwargs = {
        "add_special_tokens": False,
        "append_concat_token": False,
    },
    args = training_args
)

In [ ]:
trainer_stats = trainer.train()

In [ ]:
from unsloth import FastLanguageModel
from trl import DPOTrainer
from peft import PeftModel
from transformers import AutoTokenizer, TrainingArguments
from datasets import load_dataset
import torch

In [ ]:
max_seq_length = 1024
dtype=None
load_in_4bit = True

In [ ]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "skuma307/Gemma7b-SFT",
    max_seq_length=max_seq_length,
    dtype=dtype,
    load_in_4bit=load_in_4bit,
)
tokenizer = AutoTokenizer.from_pretrained("philschmid/gemma-tokenizer-chatml", use_fast=True)

In [ ]:
dataset = load_dataset("argilla/dpo-mix-7k")

In [ ]:
dataset

In [ ]:
column_names = list(dataset["train"].features)

In [ ]:
column_names

In [ ]:
def apply_dpo_template(example):
  if all(k in example.keys() for k in ("chosen", "rejected")):
    # For DPO, the inputs are triples of (prompt, chosen, rejected), where `chosen` and `rejected` are the final turn of a dialogue
    # We therefore need to extract the N-1 turns to form the prompt
    prompt_messages = example["chosen"][:-1]


    # Now we extract the final turn to define chosen/rejected responses
    chosen_messages = example["chosen"][-1:]
    rejected_messages = example["rejected"][-1:]
    example["text_chosen"] = tokenizer.apply_chat_template(chosen_messages, tokenize=False)
    example["text_rejected"] = tokenizer.apply_chat_template(rejected_messages, tokenize=False)
    example["text_prompt"] = tokenizer.apply_chat_template(prompt_messages, tokenize=False)
  return example

dataset = dataset.map(apply_dpo_template,remove_columns=column_names,
          desc="Formatting comparisons with prompt template",)
for split in ["train", "test"]:
    dataset[split] = dataset[split].rename_columns(
        {"text_prompt": "prompt", "text_chosen": "chosen", "text_rejected": "rejected"}
    )

In [ ]:
print(dataset)

In [ ]:
training_args = TrainingArguments(
    do_eval = True,
    evaluation_strategy = "steps",
    eval_steps = 100,
    save_strategy = "epoch",
    per_device_train_batch_size = 1,
    per_device_eval_batch_size = 16,
    gradient_accumulation_steps = 1,
    warmup_ratio = 0.1,
    num_train_epochs = 1,
    learning_rate = 5.0e-07,
    fp16 = not torch.cuda.is_bf16_supported(),
    bf16 = torch.cuda.is_bf16_supported(),
    logging_steps = 100,
    optim = "paged_adamw_8bit",
    lr_scheduler_type = "cosine",
    seed = 3507,
    output_dir = "./gemma7b_DPO_model"
)

In [ ]:
from unsloth import PatchDPOTrainer

In [ ]:
PatchDPOTrainer()

In [ ]:
trainer = DPOTrainer(
    model,
    ref_model = None,
    args= training_args,
    beta = 0.05,
    train_dataset = dataset['train'],
    eval_dataset = dataset['test'],
    tokenizer = tokenizer
)

In [ ]:
trainer.train()